In [9]:
import sys
sys.path.insert(0, "/Users/artur/workspace/recipedb")

In [10]:
from recipedb.db import get_db
import functools

In [11]:
db = get_db()

In [12]:
exs = db.allrecipes.find({"error":{"$eq":None}})

In [13]:
NUM_EXAMPLES = 10000
exs_recipes = list(map(lambda x: next(exs), range(NUM_EXAMPLES))) # grab the next NUM_EXAMPLES from cursor above

In [14]:
# Lets try simple counts first and then lets see if tf idf improves it, first step is finding common ingredients
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
# add up all the ingredients into long corpus of ingredients
corpus = functools.reduce(lambda a,b: a+b, list(map(lambda x: x['data']['ingredients'], exs_recipes)))

In [79]:
vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,5),)
tfidf = TfidfVectorizer(ngram_range=(1,5),)

In [80]:
vectorizer.fit(corpus)
tfidf.fit(corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 5), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [81]:
with open("gram.txt", "w+") as f:
    f.writelines(map(lambda x: "%s\n"%x,vectorizer.get_feature_names()))

In [82]:
with open("tfidf.txt", "w+") as f:
    f.writelines(map(lambda x: "%s\n"%x,tfidf.get_feature_names()))

In [4]:
with open("gram.txt", "r") as f:
    data = f.readlines()

In [6]:
data = list(map(lambda x: x.strip(), data))

In [23]:
# run through the list and remove the ones that occur in other ones
def remove_occurences(lst):
    i = 0
    lst = list(lst)
    while (len(lst)-1-i) > 0:
        current = lst[i] 
        nxt = lst[i+1]
        if current in nxt:
            lst.remove(current)
        else:
            i += 1
    return lst
            
            
        

In [26]:
cleaned_data = remove_occurences(data)

In [28]:
len(cleaned_data)

26673

In [29]:
len(cleaned_data)/len(exs_recipes)

2.6673